<a href="https://colab.research.google.com/github/poandpo/wanted_pre_onboarding/blob/main/%EC%9B%90%ED%8B%B0%EB%93%9C_%ED%94%84%EB%A6%AC%EC%98%A8%EB%B3%B4%EB%94%A9_%EC%BD%94%EC%8A%A4pynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 과제

In [1]:
import re

class Tokenizer():
  def __init__(self):
    self.word_dict = {'oov': 0}
    self.fit_checker = False
  
  def preprocessing(self, sequences):
    result = []
    for i in range(len(sequences)):
      tok = re.sub(r"[^a-zA-Z0-9 ]", "", sequences[i]) #영어,숫자 제외 제거
      low = tok.lower().split() # 소문자로 변환하고 분리
      result.append(low)

    return result
  
  def fit(self, sequences):
    self.fit_checker = False
    #전처리 함수 후 N차원리스트를 단순 리스트로 변환
    pre = sum(self.preprocessing(sequences), []) 
    # 중복 제외하곤 self.word_dict에 {단어: 숫자} 추가
    for word in pre:  
      if word not in self.word_dict:
         self.word_dict[word] = len(self.word_dict)
    self.fit_checker = True

  '''
  transform
  1) 토큰화 된 문장들--> 각각 문장들로 구분
  2-1) 문장의 단어들--> 단어(key)의 value(정수)로 바꿈
  2-2) 단어의 value가 존재하지 않을시 oov로 간주 --> oov 인덱스로 바뀜
  2-3) value들이 sub_re로 추가
  3) 한 문장이 다 value로 바뀌어 하나의 sub_re가 완성되면 sub_re를 result에 추가
  '''

  def transform(self, sequences):
    result = []
    tokens = self.preprocessing(sequences)
    if self.fit_checker:
      for token in tokens : #(1)
        sub_re =[]
        for word in token:
           try:
             sub_re.append(self.word_dict[word]) #(2-1)
           except KeyError:
             sub_re.append(self.word_dict['oov'])  #(2-2)
        result.append(sub_re) #(2-3)
      return result #(3)
    else:
      raise Exception("Tokenizer instance is not fitted yet.")


  def fit_transform(self, sequences):
    self.fit(sequences)
    result = self.transform(sequences)
    return result

In [2]:
from math import log 

class TfidfVectorizer:
  def __init__(self, tokenizer):
    self.tokenizer = tokenizer
    self.fit_checker = False    

    '''
    fit
    1) 토큰화하여서 나온 nested list--> 1차원 리스트로 변경 ex) [1,2,3,4]
    2) df(d,f): 각각 토큰들이 문장마다 있는지 --> 있다면 +1 
    3) idf식에 대입한 값을 result에 추가    
    '''
  def fit(self, sequences):
    tokenized = self.tokenizer.fit_transform(sequences)
    self.IDF_Matrix = []
    self.ts =  list(set(sum(tokenized, []))) # 인덱싱된 정수(토큰)들을 중복제거 후 1차원 리스트로 
    n = len(tokenized) # 총 문서
    for i in range(len(self.ts)): 
      t = self.ts[i]
      df = 0
      for token in tokenized: # (2)
        df +=  t in token
      self.IDF_Matrix.append(log(n/(df+1))) #(3)
    self.fit_checker = True

    '''
    transform
    1)TF행렬 
    1-1) 각각 문장으로 구분
    1-2) output_type을 위한 []추가 
    1-3) 특정 문서안에 특정 토큰의 갯수 TF 행렬에 추가
        cf) TF_matrix[-1] --> [-1]하지 않을시 output type 에 맞출수 없음 ex) [[],0,[],1]
    2)TF-IDF행렬: tf(d,f) X idf(d,f)
    2-1) tf(d,f): 특정 문장에서 특정 문자가 갯수() 
        idf(d,f): self.IDF_Matrix[x]
    2-2) 위 결과를 TF-IDF 행렬 추가 
    '''
  def transform(self, sequences):
    if self.fit_checker:
       tokenized = self.tokenizer.transform(sequences)
       n =len(tokenized)
       self.tfidf_matrix =[]
        # TF_행렬
       TF_matrix = []
       for i in range(n):  
         TF_matrix.append([])       
         self.tfidf_matrix.append([])
         d = tokenized[i] #특정 문서
         for x in range(len(self.ts)): #self.ts : 전체 토큰이 있는 리스트
           t = self.ts[x]  # 특정 토큰 
           TF_matrix[-1].append(d.count(t)) 

          # TF-IDF 행렬
           tfidf= d.count(t) * self.IDF_Matrix[x] 
           self.tfidf_matrix[-1].append(tfidf)
       return self.tfidf_matrix
    else:
       raise Exception("TfidfVectorizer instance is not fitted yet.")

  
  def fit_transform(self, sequences):
    self.fit(sequences)
    return self.transform(sequences)

# test

In [13]:
li=  ['I go to school.', 'I LIKE pizza!']

TOK= Tokenizer()
TOK.fit(li)

In [14]:
TOK.transform(li)

[[1, 2, 3, 4], [1, 5, 6]]

In [8]:
li2=  ['I go to school.', 'I LIKE pizza!', 'chae!!']

TOK= Tokenizer()
TOK.fit_transform(li2)

[[1, 2, 3, 4], [1, 5, 6], [7]]

In [15]:
li=  ['I go to school.', 'I LIKE pizza!']


TF=TfidfVectorizer(TOK)
TF.fit(li)

In [16]:
TF.transform(li)

[[-0.40546510810816444, 0.0, 0.0, 0.0, 0.0, 0.0],
 [-0.40546510810816444, 0.0, 0.0, 0.0, 0.0, 0.0]]

In [18]:
TF.fit_transform(li2)

[[0.0,
  0.4054651081081644,
  0.4054651081081644,
  0.4054651081081644,
  0.0,
  0.0,
  0.0],
 [0.0, 0.0, 0.0, 0.0, 0.4054651081081644, 0.4054651081081644, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.4054651081081644]]